---
title: "Debut Fztools"
format: html
date: 2025-05-07
draft: true
---

## Manage Namespaces with `StageManager`


### Register Function based on their destiny

Tidy up your function by decorating them with namespaces, so you know which variable goes to which.

In [ ]:
from fztools import StageManager

stage1 = StageManager(name="stage1")
stage2 = StageManager(name="stage2")
stage3 = StageManager(name="stage3")

input_dict = {
    "A": 1,
    "B": 2,
}
#--------------------------------
@stage1.register("A")
def plus_one(a):
    return a + 1

@stage1.register("B")
def power_two(b):
    return b ** 2
@stage1.register("DistantNode", ["A", "B"])
def make_e(a, b):
    return a + b + 3

#--------------------------------
@stage2.register("C", ["A", "B"])
def sum_all(a, b):
    return a + b
@stage2.register("A", ["A", "B"])
def sum_all(a, b):
    return a + b

#--------------------------------
@stage3.register("D", ["DistantNode","C"])
def sum_all(a, c):
    return a + c

chain = stage1 >> stage2 >> stage3

chain.to_mermaid()

### Add input data and assemble chain

You can reuse the same variable over and over, they mean different thing in different stage;

### Display dependency diagram

Create a dependency diagram of all your variables;

```mermaid
flowchart LR
    subgraph 477ab811-d8cb-4daa-9eb5-e8844ba5413c ["stage1"]
	0[/"plus_one"/]
	1[/"power_two"/]
	2[/"make_e"/]
	8[/"A"/]
	9[/"B"/]
	11[/"DistantNode"/]
	end
	subgraph b2f10fcb-4643-4740-bbf4-db4f245d0ac4 ["stage2"]
	3[/"sum_all"/]
	4[/"sum_all"/]
	10[/"C"/]
	12[/"A"/]
	end
	subgraph 8ab80742-2bd8-4010-895d-882de0b28d80 ["stage3"]
	5[/"sum_all"/]
	13[/"D"/]
	end
	subgraph 58208cf9-8e9c-4e34-80a0-f739db2387e2 ["input"]
	6[/"A"/]
	7[/"B"/]
	end
    0 --> 8
	1 --> 9
	2 --> 11
	3 --> 12
	4 --> 10
	5 --> 13
	6 --> 0
	7 --> 1
	6 --> 2
	7 --> 2
	8 --> 3
	9 --> 3
	8 --> 4
	9 --> 4
	10 --> 5
	11 -..-> 5
```

A class diagram is probably more approprate if your variables are dataframe:

### Aysnc Component Development


**Potential Graph Manipulation for Only Output Dependency**
Develop from igraph object we can derrive a dependency list of object. But the problem is there are both function objects and tangible material object, so some graph manipulation need to be done;

**Ways to Record Compuataion Status**
It may not be known if the function has completed execution or not? 
- If object are hard copy then each stage should have no output


**Identify NS and Their Stage**
This is not given directly so they needs to be inhered from the igraph object itself


**When do we not need graph manipulation?**
If we assign state to both function and object; Just function state will have to organised using something else? 


#### Use `funcs_args` property

This property is probably enough for accessing dependency tree?

In [ ]:
from functools import reduce
from colorama import Fore
# place input
chain.input = input_dict
chain.stages[1].input


# now we have a directory of objects and there relative dependency;
nei={}
for i, sg in enumerate(chain.stages):
    for k, v in sg.funcs_args.items():
        nei[(i, k)] = v
# now the function can go such as access function from a directory, it not exists 
# access by pass previous variable

# check function access
for i, Ns in nei.keys():
    chain.stages[i].funcs[Ns]

# check variable access
# the for loop just need to be something else instead


for i, Ns in nei.keys():
    deps = nei[(i, Ns)]

    # a mean to determine if the dependecy variable has been computed
    if all( chain.stages[i].input[dep] for dep in deps):
        print(Fore.GREEN + "All dependencies are met for %s %s" % (i, Ns) + Fore.RESET)

        chain.stages[i].invoke(Ns) # problem is when stage is invoke, the next input is not filled;
    else:
        print(Fore.BLUE + "Dependencies are not met! for %s %s" % (i, Ns) + Fore.RESET)
        # execute function

As a result I endup developed using this:


```
import asyncio
from asyncio import taskgroups
from fztools.stagemanager import StageManager,StageChain


class NextGenChain(StageChain):
    await_interval = 0.1
    await_maxtime = 30
    verbose = True

    def __init__(self,stages):
        self._stages = stages

    def verbose_print(self, *args, **kwargs):
        if self.verbose:
            print(*args, **kwargs)
        else:
            pass

    @property
    def dependency_dict(self):
        if not hasattr(self, "_nei"):
            self._dependency_dict = self.get_dependency_dict()
        return self._dependency_dict
    
    @property
    def deps(self):
        """shortcut for dependency_dict"""
        return self.dependency_dict

    def get_dependency_dict(self):
        """
        Get a 
        """
        nei = defaultdict(dict)
        for i, sg in enumerate(self._stages):
            sg = self._stages[i]
            func_args = set(reduce(lambda x, y: x + y, sg.funcs_args.values() ))
        
        
            for Ns, args in sg.funcs_args.items():
                nei[i][Ns] = set(args)
                # nei[(i, Ns)] = set(args)
                
                # append passing by value they are not in the previous stage
                if i > 0:
                    prev_output = set(sg.prev.funcs.keys())
                    pass_by_ns = func_args - prev_output
                    if len(pass_by_ns) > 0:
                        # print(f"Pass by {Ns}", pass_by_ns)
                        for ns in pass_by_ns:
                            nei[i-1][ns] = set([ns])

        return nei

    async def _invoke_async_(self, i, Ns):
        """ Internal Method to invoke a function asynchronously"""

        max_wait_itr = self.await_maxtime / self.await_interval

        deps = self.deps

        dep_satisified = [self.stages[i].input[dep] for dep in deps[i][Ns]]
        
        # as long as all dependency are not satisfied, wait
        while not all(dep_satisified):
            self.verbose_print(Fore.BLUE + "Waiting for dependencies for %s %s" % (i, Ns) + Fore.RESET)
            await asyncio.sleep(self.await_interval)
            # prevent iteration from infinitely runing
            max_wait_itr -= 1
            if max_wait_itr <= 0:
                self.verbose_print(Fore.RED + "Max wait reached for %s %s" % (i, Ns) + Fore.RESET)
                return None
        
        self.verbose_print(Fore.GREEN + "Invoking %s %s" % (i, Ns) + Fore.RESET)
        return self.stages[i].invoke(Ns)
    async def invoke_async(self):
        deps = self.deps
        async with asyncio.TaskGroup() as tg:
            for i in deps:
                for Ns in deps[i].keys():
                    tg.create_task(self._invoke_async_(i, Ns))
            self.verbose_print(Fore.GREEN + "All tasks created.." + Fore.RESET)
    
next_gen_chain = NextGenChain(chain._stages)

await next_gen_chain.invoke_async()
```

In [ ]:
from collections import defaultdict
from functools import reduce


self = chain

# this step is important for back fill 


nei

In [ ]:
from functools import cached_property, lru_cache
import sys
import time
class AnyClass:
    def __init__(self):
        pass

    @property
    @lru_cache(maxsize=1)
    def nei(self):
        time.sleep(1)
        return "soemthing"
    @nei.setter
    def nei(self, value):
        raise AttributeError("Cannot set attribute 'nei'")

a = AnyClass()
a.nei
a.nei
a.nei
try:
    a.nei = 2
except AttributeError as e:
    print(e)
a.nei


In [ ]:
a2 = AnyClass()
a2.nei